In [20]:
"""Task 1 Third-order letter approximation model
#To Do:
# Use them to create a model of the English language as follows. Remove any preamble and postamble. 
# Remove all characters except for (ASCII) letters (uppercase and lowercase), full stops, and spaces. 
# Make all letters uppercase.
"""

'Task 1 Third-order letter approximation model\n#To Do:\n# Use them to create a model of the English language as follows. Remove any preamble and postamble. \n# Remove all characters except for (ASCII) letters (uppercase and lowercase), full stops, and spaces. \n# Make all letters uppercase.\n'

In [21]:
import re
import os
from collections import defaultdict

In [22]:
def preprocess_text(file_path):
    # Read the file content
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
    except FileNotFoundError:
        print(f"File {file_path} not found")
        exit(1)

    # Remove preamble and postamble by finding the main text boundaries (Gutenberg's common markers)
    start = re.search(r'\*\*\* START OF THIS PROJECT GUTENBERG EBOOK .* \*\*\*', text)
    end = re.search(r'\*\*\* END OF THIS PROJECT GUTENBERG EBOOK .* \*\*\*', text)
    if start and end:
        text = text[start.end():end.start()]
    
    # Keep only uppercase letters, spaces, and periods, and make all text uppercase
    text = re.sub(r'[^A-Z. ]', '', text.upper())
    return text

In [ ]:
def create_trigram_model(text):
    # Dictionary to store trigram counts
    trigram_counts = defaultdict(int)

    # Create trigrams from the processed text
    for i in range(len(text) - 2):
        trigram = text[i:i+3]
        trigram_counts[trigram] += 1

    return trigram_counts